## Classification Problem - Cross Validation - PySpark Local

### Prepare the Data

First, import the libraries that we will need and prepare the training and test data:

In [1]:
# Import libraries
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, TrainValidationSplit
from pyspark.ml.evaluation import BinaryClassificationEvaluator

from pyspark.context import SparkContext, SparkConf
from pyspark.sql.session import SparkSession

import numpy as np

In [2]:
# PySpark session initialization
conf = SparkConf().setAppName("HPC PySpark App")

sc = SparkContext('local', conf=conf)
spark = SparkSession(sc)

# Load the source data
# csv = spark.read.csv('../../data/bank.csv', inferSchema=True, header=True, sep=';')
csv = spark.read.csv('../../data/normalized2.csv', inferSchema=True, header=True, sep=',')


# Select features and label
data = csv.select(*csv.columns[:-1], (col("y").alias("label")))
print(data)

# Split the data 70-30
splits = data.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1].withColumnRenamed("label", "trueLabel")

DataFrame[age: double, default: int, balance: double, housing: int, loan: int, duration: double, campaign: double, pdays: double, previous: double, job_admin: int, job_blue-collar: int, job_entrepreneur: int, job_housemaid: int, job_management: int, job_retired: int, job_self-employed: int, job_services: int, job_student: int, job_technician: int, job_unemployed: int, job_unknown: int, marital_divorced: int, marital_married: int, marital_single: int, education_primary: int, education_secondary: int, education_tertiary: int, education_unknown: int, contact_cellular: int, contact_telephone: int, contact_unknown: int, day_1: int, day_2: int, day_3: int, day_4: int, day_5: int, day_6: int, day_7: int, day_8: int, day_9: int, day_10: int, day_11: int, day_12: int, day_13: int, day_14: int, day_15: int, day_16: int, day_17: int, day_18: int, day_19: int, day_20: int, day_21: int, day_22: int, day_23: int, day_24: int, day_25: int, day_26: int, day_27: int, day_28: int, day_29: int, day_30: i

### Define the Pipeline
Now define a pipeline that creates a feature vector and trains a classification model

In [3]:
# Define the pipeline
assembler = VectorAssembler(inputCols = data.columns[:-1], outputCol="features")
print("Input Columns: ", assembler.getInputCols())
print("Output Column: ", assembler.getOutputCol())

lr = LogisticRegression(labelCol="label", featuresCol="features")
pipeline = Pipeline(stages=[assembler, lr])

Input Columns:  ['age', 'default', 'balance', 'housing', 'loan', 'duration', 'campaign', 'pdays', 'previous', 'job_admin', 'job_blue-collar', 'job_entrepreneur', 'job_housemaid', 'job_management', 'job_retired', 'job_self-employed', 'job_services', 'job_student', 'job_technician', 'job_unemployed', 'job_unknown', 'marital_divorced', 'marital_married', 'marital_single', 'education_primary', 'education_secondary', 'education_tertiary', 'education_unknown', 'contact_cellular', 'contact_telephone', 'contact_unknown', 'day_1', 'day_2', 'day_3', 'day_4', 'day_5', 'day_6', 'day_7', 'day_8', 'day_9', 'day_10', 'day_11', 'day_12', 'day_13', 'day_14', 'day_15', 'day_16', 'day_17', 'day_18', 'day_19', 'day_20', 'day_21', 'day_22', 'day_23', 'day_24', 'day_25', 'day_26', 'day_27', 'day_28', 'day_29', 'day_30', 'day_31', 'month_apr', 'month_aug', 'month_dec', 'month_feb', 'month_jan', 'month_jul', 'month_jun', 'month_mar', 'month_may', 'month_nov', 'month_oct', 'month_sep', 'poutcome_failure', 'pou

### Tune Parameters
You can tune parameters to find the best model for your data. A simple way to do this is to use  **TrainValidationSplit** to evaluate each combination of parameters defined in a **ParameterGrid** against a subset of the training data in order to find the best performing parameters.

#### Regularization Params and iterations
Save Regularization params and Max iteration in order to pass them at the paramGrid

In [4]:
# print(lr.explainParams())
lr_reg_params = np.logspace(-5, 5, 7)
lr_max_iter = np.linspace(1, 10, 2)

#### CrossValidation
Set CrossValidation estimator, evaluator, params and folds

In [ ]:
folds = 4
evaluator=BinaryClassificationEvaluator()
paramGrid = ParamGridBuilder().addGrid(lr.regParam, lr_reg_params).addGrid(lr.maxIter, lr_max_iter).build()

cv = CrossValidator(estimator=pipeline, evaluator=evaluator, estimatorParamMaps=paramGrid, numFolds=folds)

#### Training
Starting and ending time for the fit

In [ ]:
import time
tic = time.time()

model = cv.fit(train)

toc = time.time()
print("Elapsed time ", toc-tic)

### Test the Model
Now you're ready to apply the model to the test data.

In [ ]:
prediction = model.transform(test)
predicted = prediction.select("features", "prediction", "probability", "trueLabel")
# print(*predicted.select('prediction', 'trueLabel').collect(), sep='\n')

### Compute Confusion Matrix Metrics
Classifiers are typically evaluated by creating a *confusion matrix*, which indicates the number of:
- True Positives
- True Negatives
- False Positives
- False Negatives

From these core measures, other evaluation metrics such as *precision* and *recall* can be calculated.

In [ ]:
print('true positives:', predicted.filter('trueLabel == 1').count())
print('true negatives:', predicted.filter('trueLabel == 0').count())

In [ ]:
tp = float(predicted.filter("prediction == 1.0 AND trueLabel == 1").count())
fp = float(predicted.filter("prediction == 1.0 AND trueLabel == 0").count())
tn = float(predicted.filter("prediction == 0.0 AND trueLabel == 0").count())
fn = float(predicted.filter("prediction == 0.0 AND trueLabel == 1").count())
correctly_classified = (tp+tn) / predicted.count()

In [ ]:
metrics = spark.createDataFrame([
 ("TP", tp),
 ("FP", fp),
 ("TN", tn),
 ("FN", fn),
 ("Precision", tp / (tp + fp)),
 ("Recall", tp / (tp + fn)),
 ("Correctly Classified", correctly_classified)],["metric", "value"])
metrics.show()